# Step 4: Add a model building CI/CD pipeline

In this step you create an automated CI/CD pipeline for model building using [Amazon SageMaker Projects](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects.html). 

**From idea to production in six steps:**
||||
|---|---|---|
|1. |Experiment in a notebook ||
|2. |Scale with SageMaker AI processing jobs and SageMaker SDK ||
|3. |Operationalize with ML pipeline, model registry, and feature store ||
|4. |Add a model building CI/CD pipeline |**<<<< YOU ARE HERE**|
|5. |Add a model deployment pipeline ||
|6. |Add model and data monitoring ||

You are going to use [SageMaker AI-Provided Project Templates](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-templates-sm.html#sagemaker-projects-templates-code-commit) to provision a CI/CD workflow automation with [AWS CodePipeline](https://aws.amazon.com/codepipeline/) and a GitHub code repository.

SageMaker project templates offer you the following choice of code repositories, workflow automation tools, and pipeline stages:
- **Code repository**: Third-party Git repositories such as GitHub and Bitbucket are supported
- **CI/CD workflow automation**: AWS CodePipeline or Jenkins
- **Pipeline stages**: Model building, training, and deployment

# Step 4: 모델 구축 CI/CD 파이프라인 추가

이 단계에서는 [Amazon SageMaker Projects](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects.html)를 사용하여 모델 구축을 위한 자동화된 CI/CD 파이프라인을 생성합니다.

**아이디어에서 프로덕션까지 여섯 단계:**
||||
|---|---|---|
|1. |노트북에서 실험 ||
|2. |SageMaker AI 처리 작업과 SageMaker SDK로 확장 ||
|3. |ML 파이프라인, 모델 레지스트리, 피처 스토어로 운영화 ||
|4. |모델 구축 CI/CD 파이프라인 추가 |**<<<< 현재 위치**|
|5. |모델 배포 파이프라인 추가 ||
|6. |모델 및 데이터 모니터링 추가 ||

[SageMaker AI 제공 프로젝트 템플릿](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-templates-sm.html#sagemaker-projects-templates-code-commit)을 사용하여 [AWS CodePipeline](https://aws.amazon.com/codepipeline/)과 GitHub 코드 저장소를 통한 CI/CD 워크플로우 자동화를 구축할 것입니다.

SageMaker 프로젝트 템플릿은 다음과 같은 코드 저장소, 워크플로우 자동화 도구, 파이프라인 단계를 선택할 수 있습니다:
- **코드 저장소**: GitHub, Bitbucket 등의 서드파티 Git 저장소 지원
- **CI/CD 워크플로우 자동화**: AWS CodePipeline 또는 Jenkins
- **파이프라인 단계**: 모델 구축, 학습, 배포

<div class="alert alert-info"> Make sure you using <code>Python 3</code> kernel in JupyterLab for this notebook.</div>

In [94]:
import boto3
import sagemaker 
import json
from time import gmtime, strftime, sleep
from IPython.display import HTML

In [7]:
%store -r 

%store

try:
    initialized
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN 00-start-here notebook   ")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")

Stored variables and their in-db values:
baseline_s3_url                         -> 's3://sagemaker-us-east-1-906545278380/from-idea-t
bucket_name                             -> 'sagemaker-us-east-1-906545278380'
bucket_prefix                           -> 'from-idea-to-prod/xgboost'
dataset_feature_group_name              -> 'from-idea-to-prod-12-06-30-53'
dataset_file_local_path                 -> 'data/bank-additional/bank-additional-full.csv'
domain_id                               -> 'd-igloxuzrs3z2'
evaluation_s3_url                       -> 's3://sagemaker-us-east-1-906545278380/from-idea-t
experiment_name                         -> 'from-idea-to-prod-experiment-11-21-33-07'
feature_store_bucket_prefix             -> 'from-idea-to-prod/feature-store'
initialized                             -> True
input_s3_url                            -> 's3://sagemaker-us-east-1-906545278380/from-idea-t
mlflow_arn                              -> 'arn:aws:sagemaker:us-east-1:906545278380:mlflow

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
sm = boto3.client("sagemaker")

## Create an MLOps project

### Preparation

<div class="alert alert-info"><b>Prerequisites:</b> You need to establish the AWS CodeStar connection from your AWS account to your GitHub user or organization as described below.<br/><b>Add a tag with the key <code>sagemaker</code> and value <code>true</code> to this AWS CodeStar connection.</b></div>

For the detailed instructions see [Walk Through a SageMaker AI MLOps Project Using Third-party Git Repos](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-walkthrough-3rdgit.html) in the SageMaker Developer Guide.

### 준비

<div class="alert alert-info"><b>전제 조건:</b> 아래 설명된 대로 AWS 계정에서 GitHub 사용자 또는 조직으로의 AWS CodeStar 연결을 설정해야 합니다.<br/><b>이 AWS CodeStar 연결에 키가 <code>sagemaker</code>이고 값이 <code>true</code>인 태그를 추가하세요.</b></div>

자세한 지침은 SageMaker 개발자 가이드의 [서드파티 Git 저장소를 사용한 SageMaker AI MLOps 프로젝트 실습](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-walkthrough-3rdgit.html)을 참조하세요.

주요 준비사항:
1. AWS CodeStar와 GitHub 연동 설정
2. 필수 태그 추가 (sagemaker: true)
3. 개발자 가이드의 상세 지침 참조
4. 연결 설정 확인

이러한 준비 과정은 CI/CD 파이프라인 구축의 기초가 됩니다.

<div class="alert alert-info">You need to create two new empty private repositories in your GitHub account. One repository is for model build pipeline and another one for the model deploy pipeline. For example, you can name the repositories <code>sagemaker-ai-model-build</code> and <code>sagemaker-ai-model-deploy</code></div>

<div class="alert alert-info">GitHub 계정에 두 개의 새로운 빈 프라이빗 저장소를 생성해야 합니다. 하나는 모델 빌드 파이프라인용이고 다른 하나는 모델 배포 파이프라인용입니다. 예를 들어, 저장소 이름을 <code>sagemaker-ai-model-build</code>와 <code>sagemaker-ai-model-deploy</code>로 지정할 수 있습니다.</div>

주요 내용:
1. 두 개의 별도 GitHub 저장소 필요
   - 모델 빌드용 저장소
   - 모델 배포용 저장소
2. 저장소 특성
   - 프라이빗 저장소여야 함
   - 초기에는 비어있어야 함
3. 권장 저장소 이름
   - `sagemaker-ai-model-build`
   - `sagemaker-ai-model-deploy`

이러한 저장소 설정은 CI/CD 파이프라인의 코드 관리를 위한 기반이 됩니다.

#### Set up the GitHub connection

You need to have a GitHub personal account that you can access and connect via [AWS CodeConnection connection](https://docs.aws.amazon.com/dtconsole/latest/userguide/welcome-connections.html).

Connect to your GitHub by following the [instructions](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-walkthrough-3rdgit.html#sagemaker-proejcts-walkthrough-connect-3rdgit) to set up the GitHub connection.

Do not forget to add a tag with the key `sagemaker` and value `true` to the connection:

<img src="img/code-connection-tags.png" alt="connection tags" width="800px">

Copy the connection ARN - you need to use it as a parameter in the project creation step:

<img src="img/code-connection-arn.png" alt="connection ARN" width="800px">

#### GitHub 연결 설정

[AWS CodeConnection 연결](https://docs.aws.amazon.com/dtconsole/latest/userguide/welcome-connections.html)을 통해 접근하고 연결할 수 있는 GitHub 개인 계정이 필요합니다.

GitHub 연결을 설정하기 위해 [지침](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-walkthrough-3rdgit.html#sagemaker-proejcts-walkthrough-connect-3rdgit)에 따라 GitHub에 연결하세요.

연결에 키가 `sagemaker`이고 값이 `true`인 태그를 추가하는 것을 잊지 마세요:

<img src="img/code-connection-tags.png" alt="연결 태그" width="800px">

연결 ARN을 복사하세요 - 프로젝트 생성 단계에서 매개변수로 사용해야 합니다:

<img src="img/code-connection-arn.png" alt="연결 ARN" width="800px">

주요 단계:
1. GitHub 개인 계정 준비
2. AWS CodeConnection 설정
3. 필수 태그 추가 (sagemaker: true)
4. 연결 ARN 복사 및 보관

이 설정은 AWS와 GitHub 간의 안전하고 자동화된 연동을 가능하게 합니다.

In [62]:
# set this variable to the ARN of your code connection your created
code_connection_arn = <SET TO THE ARN OF THE CREATED CODE CONNECTION>

You can create a project programmatically in this notebook - **Option 1** or in Studio UI - **Option 2**.

Option 1 is recommended as it has no dependency on the UX</br>
Option 2 is given to demonstrate [**Create Project** UI flow](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-create.html).

프로젝트를 생성하는 방법으로 두 가지 옵션이 있습니다:
- **옵션 1**: 이 노트북에서 프로그래밍 방식으로 생성
- **옵션 2**: Studio UI에서 생성

옵션 1이 UI에 대한 의존성이 없어 권장됩니다.</br>
옵션 2는 [**프로젝트 생성** UI 흐름](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-create.html)을 보여주기 위해 제공됩니다.

주요 특징:
1. 옵션 1 (프로그래밍 방식)
   - UI 독립적
   - 자동화 용이
   - 반복 가능한 설정
   - 버전 관리 가능

2. 옵션 2 (UI 방식)
   - 시각적 인터페이스
   - 단계별 가이드
   - 직관적 설정
   - 학습 목적에 유용

### Option 1: Create project programmatically - recommended
In this section you use `boto3` to create an MLOps project via a SageMaker API.

### 옵션 1: 프로그래밍 방식으로 프로젝트 생성 - 권장
이 섹션에서는 `boto3`를 사용하여 SageMaker API를 통해 MLOps 프로젝트를 생성합니다.

주요 내용:
1. boto3 라이브러리 활용
2. SageMaker API를 통한 프로젝트 생성
3. 프로그래밍 방식의 자동화된 설정
4. 재현 가능한 프로젝트 구성

이 방식의 장점:
- 코드로 관리되는 인프라
- 자동화된 프로세스
- 일관된 설정 가능
- 버전 관리 용이

In [63]:
sc = boto3.client("servicecatalog")

sc_provider_name = "Amazon SageMaker"
sc_product_name = "MLOps template for model building, training, and deployment with third-party Git repositories using CodePipeline"

In [64]:
# find a Service Catalog product with the specific SageMaker project template
p_ids = [p['ProductId'] for p in sc.search_products(
    Filters={
        'FullTextSearch': [sc_product_name]
    },
)['ProductViewSummaries'] if p["Name"]==sc_product_name]

In [65]:
p_ids

['prod-cwbsact3annui']

In [66]:
# If you get any exception from this code, go to the Option 2 and create a project in Studio UI
if not len(p_ids):
    raise Exception("No Amazon SageMaker ML Ops products found!")
elif len(p_ids) > 1:
    raise Exception("Too many matching Amazon SageMaker ML Ops products found!")
else:
    product_id = p_ids[0]
    print(f"ML Ops product id: {product_id}")

ML Ops product id: prod-cwbsact3annui


In [67]:
# output what this project is about
sc.describe_product(Id=product_id)['ProductViewSummary']['ShortDescription']

'Use this template to automate the entire model lifecycle that includes both model buidling and deployment workflows. Ideally suited for continuous integration and continuous deployment (CI/CD) of ML models. Process data, extract features, train and test models, and register them in the model registry. Attach your own Git repository to the project for checking in and managing code versions. Kick off the model deployment workflow by approving the model registered in the model registry for deployment either manually or automatically. You can customize the seed code and the configuration files to suit your requirements. AWS CodePipeline is used to orchestrate the model deployment.\n\nModel building pipeline: SageMaker Pipelines\nCode repository: Third party Git\nOrchestration: AWS CodePipeline\n'

In [68]:
# get the latest template version
provisioning_artifact_id = sorted(
    [i for i in sc.list_provisioning_artifacts(
        ProductId=product_id
    )['ProvisioningArtifactDetails'] if i['Guidance']=='DEFAULT'],
    key=lambda d: d['Name'], reverse=True)[0]['Id']

In [69]:
provisioning_artifact_id

'pa-jt7oyaklc3eym'

In [70]:
sc.describe_provisioning_artifact(ProductId=product_id, ProvisioningArtifactId=provisioning_artifact_id)

{'ProvisioningArtifactDetail': {'Id': 'pa-jt7oyaklc3eym',
  'Name': 'v2.0',
  'Description': 'Adding error handling for access denied for seed code checkin',
  'Type': 'CLOUD_FORMATION_TEMPLATE',
  'CreatedTime': datetime.datetime(2024, 12, 10, 5, 6, 31, tzinfo=tzlocal()),
  'Active': True,
  'Guidance': 'DEFAULT'},
 'Info': {'TemplateUrl': 'https://s3.us-east-1.amazonaws.com/ciclo-us-east-1-prod-product-templates/model_build_deploy_toolchain_3p_git_template.yml'},
 'Status': 'AVAILABLE',
 'ResponseMetadata': {'RequestId': '1166418b-fb61-4e1b-b6a4-9cb05bd4b008',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1166418b-fb61-4e1b-b6a4-9cb05bd4b008',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '414',
   'date': 'Fri, 14 Feb 2025 11:02:36 GMT'},
  'RetryAttempts': 0}}

In [75]:
# set unique project name
project_name = f"mlops-{strftime('%m-%d-%H-%M-%S', gmtime())}"

Set the parameters of the project template. Set the variables `model_build_code_repository_full_name` and `model_deploy_code_repository_full_name` to the full names of your GitHub repositories. These must be new empty repositories.

In [76]:
# Branch name for the Model Building and Training Code Repository
model_build_code_repository_branch = 'main'
# Full repository name of the Model Building and Training Code Repository, which would be username/reponame or organizationname/reponame
model_build_code_repository_full_name = <ENTER YOUR FULL GITHUB REPO FOR MODEL BUILD NAME HERE> # e.g. username/reponame

# Branch name for the Model Deployment Code Repository
model_deploy_code_repository_branch = 'main'
# Full repository name of the Model Deployment Code Repository, which would be username/reponame or organizationname/reponame
model_deploy_code_repository_full_name = <ENTER YOUR FULL GITHUB REPO FOR MODEL DEPLOY NAME HERE> # e.g. username/reponame

In [77]:
# set project parameters
project_parameters = [
    {
        'Key': 'ModelBuildCodeRepositoryBranch',
        'Value': model_build_code_repository_branch,
    },
    {
        'Key': 'ModelBuildCodeRepositoryFullname',
        'Value': model_build_code_repository_full_name,
    },
    {
        'Key': 'ModelDeployCodeRepositoryBranch',
        'Value': model_deploy_code_repository_branch,
    },
    {
        'Key': 'ModelDeployCodeRepositoryFullname',
        'Value': model_deploy_code_repository_full_name,
    },
        {
        'Key': 'CodeConnectionArn',
        'Value': code_connection_arn,
    },
]

Finally, create a SageMaker project from the service catalog product template:

In [91]:
print(f'''Creating a {project_name} using {sc_product_name} with the following parameters:
{json.dumps(project_parameters, indent=2)}
''')

Creating a mlops-02-14-11-03-33 using MLOps template for model building, training, and deployment with third-party Git repositories using CodePipeline with the following parameters:
[
  {
    "Key": "ModelBuildCodeRepositoryBranch",
    "Value": "main"
  },
  {
    "Key": "ModelBuildCodeRepositoryFullname",
    "Value": "yevgeniyilyin/sagemaker-ai-model-build-2"
  },
  {
    "Key": "ModelDeployCodeRepositoryBranch",
    "Value": "main"
  },
  {
    "Key": "ModelDeployCodeRepositoryFullname",
    "Value": "yevgeniyilyin/sagemaker-ai-model-deploy-2"
  },
  {
    "Key": "CodeConnectionArn",
    "Value": "arn:aws:codeconnections:us-east-1:906545278380:connection/f76f091f-f02a-4390-8abd-184d1735ca3a"
  }
]



In [78]:
# create SageMaker project
r = sm.create_project(
    ProjectName=project_name,
    ProjectDescription="Model build and deploy project",
    ServiceCatalogProvisioningDetails={
        'ProductId': product_id,
        'ProvisioningArtifactId': provisioning_artifact_id,
        'ProvisioningParameters': project_parameters
    },
)

print(r)
project_id = r["ProjectId"]

{'ProjectArn': 'arn:aws:sagemaker:us-east-1:906545278380:project/mlops-02-14-11-03-33', 'ProjectId': 'p-ca7phmcraepa', 'ResponseMetadata': {'RequestId': '954c8ede-e181-4959-8ed0-f04946e002fb', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '954c8ede-e181-4959-8ed0-f04946e002fb', 'content-type': 'application/x-amz-json-1.1', 'content-length': '115', 'date': 'Fri, 14 Feb 2025 11:03:38 GMT'}, 'RetryAttempts': 0}}


<div class="alert alert-info">Wait until project creation is completed by running the next cell</div>

In [84]:
# Project creation takes about 3-5 min
while sm.describe_project(ProjectName=project_name)['ProjectStatus'] not in ['CreateCompleted', 'CreateFailed']:
    print("Waiting for project creation completion")
    sleep(10)
    
print(f"MLOps project {project_name} creation completed")

MLOps project mlops-02-14-11-03-33 creation completed


In [85]:
assert sm.describe_project(ProjectName=project_name)['ProjectStatus'] == 'CreateCompleted', 'Project status must be CreateCompleted!'

### End of Option 1: Create project programmatically
Now you have provisioned a project template in your SageMaker environment. Navigate to the section **Configure the MLOps project**.

---

### 옵션 1: 프로그래밍 방식 프로젝트 생성 완료
이제 SageMaker 환경에 프로젝트 템플릿을 프로비저닝했습니다. **MLOps 프로젝트 구성** 섹션으로 이동하세요.

---

주요 사항:
1. 프로젝트 템플릿 프로비저닝 완료
2. SageMaker 환경에 설정 완료
3. 다음 단계인 MLOps 프로젝트 구성으로 진행
4. 자동화된 설정이 성공적으로 완료됨

이제 프로젝트의 기본 구조가 준비되어 구체적인 MLOps 구성을 시작할 수 있습니다.

### Option 2: Create a project in Studio UI
<div class="alert alert-info"><b>Skip this section if you have arealdy created a project programmatically</b></div>

<div class="alert alert-info">You need to complete the <b>Preparation</b> step before creating an MLOps project</div>

### 옵션 2: Studio UI에서 프로젝트 생성
<div class="alert alert-info"><b>이미 프로그래밍 방식으로 프로젝트를 생성했다면 이 섹션을 건너뛰세요</b></div>

<div class="alert alert-info">MLOps 프로젝트를 생성하기 전에 <b>준비</b> 단계를 완료해야 합니다</div>

주요 안내:
1. 중복 작업 방지
   - 프로그래밍 방식(옵션 1)으로 이미 생성했다면 이 섹션 불필요
2. 사전 준비 필수
   - 준비 단계 완료가 MLOps 프로젝트 생성의 전제 조건
3. UI 기반 생성
   - Studio UI를 통한 시각적 프로젝트 설정 방법 제공

이 옵션은 시각적 인터페이스를 선호하는 사용자를 위한 대체 방법입니다.

Follow the [instructions for **Step 2: Create the Project**](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-walkthrough-3rdgit.html#sagemaker-proejcts-walkthrough-create-3rdgit) in the Developer Guide to create an MLOps CI/CD project.

For the template choose the **Model building, training, and deployment with third-party Git repositories using CodePipeline**:

![](img/mlops-project-name.png)

Wait until project is created.

[개발자 가이드의 **2단계: 프로젝트 생성**](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-walkthrough-3rdgit.html#sagemaker-proejcts-walkthrough-create-3rdgit) 지침에 따라 MLOps CI/CD 프로젝트를 생성하세요.

템플릿으로는 **CodePipeline을 사용하여 서드파티 Git 저장소로 모델 구축, 학습 및 배포**를 선택하세요:

![](img/mlops-project-name.png)

프로젝트가 생성될 때까지 기다리세요.

주요 단계:
1. 개발자 가이드의 지침 따르기
2. 적절한 템플릿 선택
   - 서드파티 Git 저장소 활용
   - CodePipeline 기반 워크플로우
3. 프로젝트 생성 대기
   - 완전한 설정을 위한 시간 필요
   - 자동화된 리소스 프로비저닝 진행

이 설정은 완전한 MLOps 파이프라인의 기반이 됩니다.

### Resolve issues with project creation

#### Error messages
❗ If you see an error message similar to:
```
Your project couldn't be created
Studio encountered an error when creating your project. Try recreating the project again.

CodeBuild is not authorized to perform: sts:AssumeRole on arn:aws:iam::XXXX:role/service-role/AmazonSageMakerServiceCatalogProductsCodeBuildRole (Service: AWSCodeBuild; Status Code: 400; Error Code: InvalidInputException; Request ID: 4cf59a54-0c59-476a-a970-0ac656db4402; Proxy: null)
```

see steps 5-6 of [SageMaker Studio Permissions Required to Use Projects](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-studio-updates.html). Make sure you have all required project roles listed in the **Apps** card under **Projects**. 

Alternatively, you can create the required roles by using the provided CloudFormation template [`cfn-templates/sagemaker-project-templates-roles.yaml`](cfn-templates/sagemaker-project-templates-roles.yaml). 
Run in the repository clone directory from the command line terminal where you have the corresponding permissions:

```sh
aws cloudformation deploy \
    --template-file cfn-templates/sagemaker-project-templates-roles.yaml \
    --stack-name sagemaker-project-template-roles \
    --capabilities CAPABILITY_IAM CAPABILITY_NAMED_IAM \
    --parameter-overrides \
    CreateCloudFormationRole=YES \
    CreateCodeBuildRole=YES \
    CreateCodePipelineRole=YES \
    CreateEventsRole=YES \
    CreateProductsExecutionRole=YES 
```

### 프로젝트 생성 관련 문제 해결

#### 오류 메시지
❗ 다음과 같은 오류 메시지가 표시되는 경우:
```
프로젝트를 생성할 수 없습니다
Studio에서 프로젝트 생성 중 오류가 발생했습니다. 프로젝트를 다시 생성해 보세요.

CodeBuild가 arn:aws:iam::XXXX:role/service-role/AmazonSageM**********************uctsCodeBuildRole에 대해 sts:AssumeRole을 수행할 권한이 없습니다 (Service: AWSCodeBuild; Status Code: 400; Error Code: InvalidInputException; Request ID: 4cf59a54-0c59-476a-a970-0ac656db4402; Proxy: null)
```

[프로젝트 사용에 필요한 SageMaker Studio 권한](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-studio-updates.html)의 5-6단계를 참조하세요. **Projects** 아래 **Apps** 카드에 필요한 모든 프로젝트 역할이 있는지 확인하세요.

또는 제공된 CloudFormation 템플릿 [`cfn-templates/sagemaker-project-templates-roles.yaml`](cfn-templates/sagemaker-project-templates-roles.yaml)을 사용하여 필요한 역할을 생성할 수 있습니다.
해당 권한이 있는 명령줄 터미널의 저장소 복제 디렉토리에서 다음을 실행하세요:

```sh
aws cloudformation deploy \
    --template-file cfn-templates/sagemaker-project-templates-roles.yaml \
    --stack-name sagemaker-project-template-roles \
    --capabilities CAPABILITY_IAM CAPABILITY_NAMED_IAM \
    --parameter-overrides \
    CreateCloudFormationRole=YES \
    CreateCodeBuildRole=YES \
    CreateCodePipelineRole=YES \
    CreateEventsRole=YES \
    CreateProductsExecutionRole=YES 
```

주요 해결 방법:
1. 필요한 권한 확인
2. 누락된 역할 확인
3. CloudFormation을 통한 역할 생성
4. 모든 필수 권한이 올바르게 설정되었는지 검증

### End of Option 2: Create a project in Studio UI
Now when you have the project created, move to the section **Configure the MLOps project**.

---

### 옵션 2: Studio UI에서 프로젝트 생성 완료
이제 프로젝트가 생성되었으니, **MLOps 프로젝트 구성** 섹션으로 이동하세요.

---

주요 사항:
1. UI를 통한 프로젝트 생성 완료
2. 기본 설정 완료됨
3. 다음 단계로 진행 준비
4. MLOps 프로젝트의 상세 구성 시작

이제 프로젝트의 기본 구조가 준비되어 구체적인 MLOps 구성을 시작할 수 있습니다. 이어지는 섹션에서는 프로젝트의 세부적인 설정과 구성을 다룰 것입니다.

## Configure the MLOps project
The provisioned MLOps project implements two CodePipeline pipelines. One is for model building and another for model deployment. The project also populates two GitHub repositories your specified with seed code. This notebook explains and configures the model building pipeline. The next notebook `05-deploy` deals with the second part - the model deployment pipeline.

## MLOps 프로젝트 구성
프로비저닝된 MLOps 프로젝트는 두 개의 CodePipeline 파이프라인을 구현합니다. 하나는 모델 구축용이고 다른 하나는 모델 배포용입니다. 또한 프로젝트는 지정한 두 개의 GitHub 저장소에 시드 코드를 채워넣습니다. 이 노트북은 모델 구축 파이프라인을 설명하고 구성합니다. 다음 노트북인 `05-deploy`는 두 번째 부분인 모델 배포 파이프라인을 다룹니다.

주요 내용:
1. 두 개의 독립적인 파이프라인:
   - 모델 구축 파이프라인
   - 모델 배포 파이프라인
2. GitHub 저장소 자동 설정:
   - 시드 코드 자동 배포
   - 기본 구조 제공
3. 현재 노트북의 범위:
   - 모델 구축 파이프라인 구성에 집중
4. 다음 단계:
   - 모델 배포 파이프라인은 별도 노트북에서 다룸

### Model building pipeline

The project provisions and runs a default model building pipeline automatically as soon as it has been created. This pipeline is a sample placeholder in the project for your own custom pipeline. Ignore the default pipeline for the moment.
The project templates deploys the following resources in your AWS account:

![](img/mlops-model-build-train.png)

The main components are:
1. The project template is made available through SageMaker Projects and AWS Service Catalog portfolio
2. A CodePipeline pipeline with two stages - `Source` to download the source code from your GitHub repository and `Build` to create and execute a SageMaker pipeline
3. A default SageMaker pipeline with model build, train, and register workflow
4. A seed code repository in GitHub with a provided default version of a seed code

### 모델 구축 파이프라인

프로젝트가 생성되자마자 기본 모델 구축 파이프라인이 자동으로 프로비저닝되고 실행됩니다. 이 파이프라인은 사용자 정의 파이프라인을 위한 샘플 자리표시자입니다. 지금은 기본 파이프라인을 무시하세요.
프로젝트 템플릿은 AWS 계정에 다음과 같은 리소스를 배포합니다:

![](img/mlops-model-build-train.png)

주요 구성 요소:
1. 프로젝트 템플릿은 SageMaker Projects와 AWS Service Catalog 포트폴리오를 통해 제공됨
2. 두 단계로 구성된 CodePipeline 파이프라인
   - `Source`: GitHub 저장소에서 소스 코드 다운로드
   - `Build`: SageMaker 파이프라인 생성 및 실행
3. 모델 구축, 학습, 등록 워크플로우가 포함된 기본 SageMaker 파이프라인
4. 기본 버전의 시드 코드가 제공되는 GitHub의 시드 코드 저장소

이러한 구성은 완전한 MLOps 워크플로우의 기초를 제공합니다.

This project contains all the required code and the insfrastructure to implement an automated CI/CD pipeline from a pre-defined template. 
To start using the project with your pipeline, you need to complete the following steps:
1. Clone the project GitHub repository to your notebook local storage
2. Replace the ML pipeline template sample code with your actual pipeline construction code, as implemented in the step 3 notebook
3. Modify the `codebuild-buildspec.yml` file to reference the correct Python module name and to set project parameters

Next sections guide you through these steps. For detailed instructions and a hands-on example, refer to the development guide [SageMaker MLOps Project Walkthrough](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-walkthrough-3rdgit.html).

If you used the option 1 `boto3` to create an MLOps project, the `project_name` and `project_id` are set automatically. You can run the following code cell to print the values. If you followed the UI instructions to create a project, you must set the `project_name` manually.

이 프로젝트는 사전 정의된 템플릿에서 자동화된 CI/CD 파이프라인을 구현하는 데 필요한 모든 코드와 인프라를 포함하고 있습니다.
파이프라인과 함께 프로젝트를 시작하려면 다음 단계를 완료해야 합니다:

1. 프로젝트 GitHub 저장소를 노트북 로컬 스토리지로 복제
2. ML 파이프라인 템플릿 샘플 코드를 3단계 노트북에서 구현한 실제 파이프라인 구성 코드로 교체
3. `codebuild-buildspec.yml` 파일을 수정하여 올바른 Python 모듈 이름을 참조하고 프로젝트 매개변수 설정

다음 섹션에서 이러한 단계들을 안내합니다. 자세한 지침과 실습 예제는 개발자 가이드 [SageMaker MLOps 프로젝트 실습](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-walkthrough-3rdgit.html)을 참조하세요.

옵션 1 `boto3`를 사용하여 MLOps 프로젝트를 생성한 경우, `project_name`과 `project_id`가 자동으로 설정됩니다. 다음 코드 셀을 실행하여 값을 출력할 수 있습니다. UI 지침에 따라 프로젝트를 생성한 경우에는 `project_name`을 수동으로 설정해야 합니다.

주요 포인트:
1. 완전한 CI/CD 환경 제공
2. 단계별 설정 과정
3. 상세 문서 참조 가능
4. 프로젝트 식별자 설정 방법

In [122]:
try:
    print(project_name)
    print(project_id)
    print(model_build_code_repository_full_name)
    print(code_connection_arn)
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("Set the code_connection_arn, project_name, and repository full name in the following code cell")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

mlops-02-14-11-03-33
p-ca7phmcraepa
yevgeniyilyin/sagemaker-ai-model-build-2
arn:aws:codeconnections:us-east-1:906545278380:connection/f76f091f-f02a-4390-8abd-184d1735ca3a


In [187]:
# project_name = "<SET TO THE NAME OF THE CREATED PROJECT>" # Keep commented out if you used option 1 to create a project
# model_build_code_repository_full_name = "<SET TO THE FULL NAME OF MODEL BUILD REPO>" # Keep commented out if you used option 1 to create a project

r = sm.describe_project(ProjectName=project_name)
project_id = r['ProjectId']
project_arn = r['ProjectArn']
repository_name = model_build_code_repository_full_name.split('/')[1]
git_folder = project_name
project_folder = f'sagemaker-{project_id}-modelbuild'
project_path = f'{git_folder}/{project_folder}'

%store project_name
%store project_arn
%store project_id
    
print(f"Project path: {project_path}")

Stored 'project_name' (str)
Stored 'project_arn' (str)
Stored 'project_id' (str)
Project path: mlops-02-14-11-03-33/sagemaker-p-ca7phmcraepa-modelbuild


### Explore the project in the Studio UI

Click on the link constructed by the following code cell. Explore the project and project components in the Studio UI.

### Studio UI에서 프로젝트 탐색

다음 코드 셀에서 생성된 링크를 클릭하세요. Studio UI에서 프로젝트와 프로젝트 구성 요소를 탐색하세요.

주요 탐색 포인트:
1. 프로젝트 개요 확인
2. 구성 요소 살펴보기:
   - 파이프라인 구성
   - 리소스 설정
   - 워크플로우 구조
3. UI를 통한 시각적 이해:
   - 프로젝트 구조
   - 구성 요소 간 관계
   - 설정 상태

Studio UI를 통해 프로젝트의 전체적인 구조와 설정을 직관적으로 파악할 수 있습니다.

In [105]:
# Show the project link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/projects/{}/">the project</a> in the Studio UI</b>'.format(
            domain_id, region, project_name))
)

### 1. Clone the project seed code to the JupyterLab file system
You need to clone the project code from the GitHub repository by using JupyterLab terminal and GitHub CLI.

1. Open a new terminal window via **File** > **New** > **Terminal**
2. Copy the output of the following sell and paste it into the terminal command line
3. Clone your model build repository using `git clone` with `codeconnections` credential helper.

### 1. 프로젝트 시드 코드를 JupyterLab 파일 시스템으로 복제
JupyterLab 터미널과 GitHub CLI를 사용하여 GitHub 저장소에서 프로젝트 코드를 복제해야 합니다.

1. **File** > **New** > **Terminal**을 통해 새 터미널 창 열기
2. 다음 셸의 출력을 복사하여 터미널 명령줄에 붙여넣기
3. `codeconnections` 자격 증명 도우미를 사용하여 `git clone` 명령으로 모델 빌드 저장소 복제

주요 단계:
1. 터미널 접근
   - JupyterLab의 메뉴 사용
   - 새 터미널 세션 시작

2. 명령어 준비
   - 제공된 스크립트 복사
   - 터미널에 붙여넣기 준비

3. 저장소 복제
   - git clone 명령 실행
   - AWS CodeConnection 인증 활용
   - 로컬 시스템에 코드 다운로드

In [158]:
def copy_output(text):
    return HTML(f'''
        <div style="position: relative;">
            <pre>{text}</pre>
            <button onclick="navigator.clipboard.writeText(`{text}`)"
                    style="position: absolute; top: 5px; right: 5px;">
                Copy
            </button>
        </div>
    ''')
    
cmd = f'''
git config --global credential.UseHttpPath true
git config --global credential.helper 'cache --timeout=720000'
git config --global credential.helper '!aws codecommit credential-helper $@'

mkdir -p $HOME/{project_path}

git clone https://codeconnections.{region}.amazonaws.com/git-http/{code_connection_arn.split(':')[4]}/{region}/{code_connection_arn.split(':')[-1].split('/')[-1]}/{model_build_code_repository_full_name}.git  $HOME/{project_path}
'''

copy_output(cmd)

### 2. Replace pipeline construction code

The following steps are required to customize the project which contains the seed code. The next code cell executes all the required steps, you don't need to do anything manually. The following text is for your information only.

- The seed source code is in the folder `<project_name>/sagemaker-<project-id>-modelbuild`.
- The original file `codebuild-buildspec.yml` is renamed to `codebuild-buildspec-original.yml`.
- MLOps project's code repository folder containing the pipeline code is renamed from `abalone` folder to `fromideatoprod`.
- The original file with the template pipeline `pipeline.py` is renamed to `pipeline-original.py`.
- Copy the `pipeline_steps` Python modules to the `pipelines` folder in the MLOps project's code repository folder.
- Copy the `requirements.txt` created in the notebook 3 to the `pipelines` folder in the MLOps project's code repository folder.
- Copy SageMaker Python SDK default configuration file `config.yaml` from the notebook 3 to the `pipelines` folder in the MLOps project's code repository folder.

다음 단계들은 시드 코드가 포함된 프로젝트를 커스터마이즈하는 데 필요합니다. 다음 코드 셀이 모든 필요한 단계를 실행하므로, 수동으로 할 필요가 없습니다. 다음 내용은 참고용입니다.

- 시드 소스 코드는 `<project_name>/sagemaker-<project-id>-modelbuild` 폴더에 있습니다.
- 원본 파일 `codebuild-buildspec.yml`이 `codebuild-buildspec-original.yml`로 이름이 변경됩니다.
- 파이프라인 코드가 포함된 MLOps 프로젝트의 코드 저장소 폴더가 `abalone` 폴더에서 `fromideatoprod`로 이름이 변경됩니다.
- 템플릿 파이프라인이 있는 원본 파일 `pipeline.py`가 `pipeline-original.py`로 이름이 변경됩니다.
- `pipeline_steps` Python 모듈을 MLOps 프로젝트의 코드 저장소 폴더의 `pipelines` 폴더로 복사합니다.
- 노트북 3에서 생성된 `requirements.txt`를 MLOps 프로젝트의 코드 저장소 폴더의 `pipelines` 폴더로 복사합니다.
- 노트북 3의 SageMaker Python SDK 기본 구성 파일 `config.yaml`을 MLOps 프로젝트의 코드 저장소 폴더의 `pipelines` 폴더로 복사합니다.

주요 작업:
1. 파일 구조 재구성
2. 파일 이름 변경
3. 필요 파일 복사
4. 구성 파일 설정

이 모든 과정이 자동으로 실행되어 프로젝트를 사용자의 필요에 맞게 구성합니다.

In [161]:
# see the workshop folder name
!pwd

/home/sagemaker-user/amazon-sagemaker-from-idea-to-production


In [162]:
# if you local path for the workshop folder is different, set the correct absolute path to the variable workshop_folder
workshop_folder = "amazon-sagemaker-from-idea-to-production"

In [194]:
!mkdir -p ~/{workshop_folder}/pipelines
!mv ~/{project_path}/codebuild-buildspec.yml ~/{project_path}/codebuild-buildspec-original.yml
!mv ~/{project_path}/setup.py ~/{project_path}/setup-original.py
!mv ~/{project_path}/pipelines/abalone ~/{project_path}/pipelines/fromideatoprod
!mv ~/{project_path}/pipelines/fromideatoprod/pipeline.py ~/{project_path}/pipelines/fromideatoprod/pipeline-original.py
!cp ~/{workshop_folder}/pipeline_steps/* ~/{project_path}/pipelines/
!cp ~/{workshop_folder}/pipeline_steps/* ~/{workshop_folder}/pipelines/
!cp ~/{workshop_folder}/requirements.txt ~/{project_path}
!cp ~/{workshop_folder}/config.yaml ~/{project_path}

mv: cannot stat '/home/sagemaker-user/mlops-02-14-11-03-33/sagemaker-p-ca7phmcraepa-modelbuild/pipelines/abalone': No such file or directory
cp: -r not specified; omitting directory '/home/sagemaker-user/amazon-sagemaker-from-idea-to-production/pipeline_steps/__pycache__'
cp: -r not specified; omitting directory '/home/sagemaker-user/amazon-sagemaker-from-idea-to-production/pipeline_steps/__pycache__'


Execute the following cell to write pipeline construction code to the file `pipeline.py`. Re-use the code from the step 3 notebook as the function `get_pipeline()`.

<div style="border: 4px solid coral; text-align: center; margin: auto;">
    <p style=" text-align: center; margin: auto;">The pipeline construction code works with both S3 raw input dataset or feature set from the Feature Store, you need to pass a corresponding input parameter to the pipeline.
    </p>
</div>

다음 셀을 실행하여 파이프라인 구성 코드를 `pipeline.py` 파일에 작성하세요. 3단계 노트북의 코드를 `get_pipeline()` 함수로 재사용합니다.

<div style="border: 4px solid coral; text-align: center; margin: auto;">
    <p style=" text-align: center; margin: auto;">파이프라인 구성 코드는 S3 원시 입력 데이터셋 또는 Feature Store의 피처셋 모두와 작동합니다. 해당하는 입력 매개변수를 파이프라인에 전달해야 합니다.
    </p>
</div>

주요 포인트:
1. 파이프라인 코드 재사용
   - 3단계 노트북의 코드 활용
   - get_pipeline() 함수로 구성

2. 데이터 소스 유연성
   - S3 원시 데이터 지원
   - Feature Store 피처셋 지원
   - 입력 매개변수로 선택 가능

3. 파일 생성
   - pipeline.py 파일에 코드 작성
   - 자동화된 파이프라인 구성 가능

이를 통해 유연하고 재사용 가능한 ML 파이프라인을 구성할 수 있습니다.

In [195]:
%%writefile pipeline.py

import pandas as pd
import json
import boto3
import pathlib
import io
import os
import sagemaker
import mlflow
from time import gmtime, strftime, sleep
from sagemaker.deserializers import CSVDeserializer
from sagemaker.serializers import CSVSerializer

from sagemaker.workflow.execution_variables import ExecutionVariables
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.xgboost.estimator import XGBoost
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import (
    ProcessingInput, 
    ProcessingOutput, 
    ScriptProcessor
)
from sagemaker.inputs import TrainingInput

from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import (
    ProcessingStep, 
    TrainingStep, 
    CreateModelStep,
    CacheConfig
)
from sagemaker.workflow.check_job_config import CheckJobConfig
from sagemaker.workflow.parameters import (
    ParameterInteger, 
    ParameterFloat, 
    ParameterString, 
    ParameterBoolean
)
from sagemaker.workflow.quality_check_step import (
    DataQualityCheckConfig,
    ModelQualityCheckConfig,
    QualityCheckStep,
)
from sagemaker.workflow.clarify_check_step import (
    ModelBiasCheckConfig, 
    ClarifyCheckStep, 
    ModelExplainabilityCheckConfig
)
from sagemaker import Model
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.model_step import ModelStep
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.conditions import (
    ConditionGreaterThan,
    ConditionGreaterThanOrEqualTo
)
from sagemaker.workflow.parallelism_config import ParallelismConfiguration
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import (
    Join,
    JsonGet
)
from sagemaker.workflow.lambda_step import (
    LambdaStep,
    LambdaOutput,
    LambdaOutputTypeEnum,
)
from sagemaker.lambda_helper import Lambda

from sagemaker.model_metrics import (
    MetricsSource, 
    ModelMetrics, 
    FileSource
)
from sagemaker.drift_check_baselines import DriftCheckBaselines
from sagemaker.workflow.pipeline_definition_config import PipelineDefinitionConfig 
from sagemaker.image_uris import retrieve
from sagemaker.workflow.function_step import step
from sagemaker.workflow.step_outputs import get_step
from sagemaker.model_monitor import DatasetFormat, model_monitoring

from pipelines.preprocess import preprocess
from pipelines.evaluate import evaluate
from pipelines.register import register
from pipelines.extract import prepare_datasets

def get_sagemaker_client(region):
     return boto3.Session(region_name=region).client("sagemaker")

def get_pipeline_session(region, bucket_name):
    """Gets the pipeline session based on the region.

    Args:
        region: the aws region to start the session
        bucket_name: the bucket to use for storing the artifacts

    Returns:
        PipelineSession instance
    """

    boto_session = boto3.Session(region_name=region)
    sagemaker_client = boto_session.client("sagemaker")

    return PipelineSession(
        boto_session=boto_session,
        sagemaker_client=sagemaker_client,
        default_bucket=bucket_name,
    )

def get_pipeline_custom_tags(new_tags, region, sagemaker_project_name=None):
    try:
        print(f"Getting project tags for {sagemaker_project_name}")
        
        sm_client = get_sagemaker_client(region)
        
        project_arn = sm_client.describe_project(ProjectName=sagemaker_project_name)['ProjectArn']
        project_tags = sm_client.list_tags(ResourceArn=project_arn)['Tags']

        print(f"Project tags: {project_tags}")
        
        for project_tag in project_tags:
            new_tags.append(project_tag)
            
    except Exception as e:
        print(f"Error getting project tags: {e}")
        
    return new_tags
    
def get_pipeline(
    region,
    sagemaker_project_id=None,
    sagemaker_project_name=None,
    role=None,
    bucket_name=None,
    bucket_prefix="from-idea-to-prod/xgboost",
    input_s3_url=None,
    feature_group_name=None,
    model_package_group_name="from-idea-to-prod-model-group",
    pipeline_name_prefix="from-idea-to-prod-pipeline",
    process_instance_type="ml.m5.large",
    train_instance_type="ml.m5.xlarge",
    test_score_threshold=0.70,
    tracking_server_arn=None,
):
    """Gets a SageMaker ML Pipeline instance.
    
    Returns:
        an instance of a pipeline
    """
    if feature_group_name is None and input_s3_url is None:
        print("One of feature_group_name or input_s3_url must be provided. Exiting...")
        return None

    session = get_pipeline_session(region, bucket_name)
    sm = session.sagemaker_client
    
    if role is None:
        role = sagemaker.session.get_execution_role(session)

    print(f"sagemaker version: {sagemaker.__version__}")
    print(f"Execution role: {role}")
    print(f"Input S3 URL: {input_s3_url}")
    print(f"Feature group: {feature_group_name}")
    print(f"Model package group: {model_package_group_name}")
    print(f"Pipeline name prefix: {pipeline_name_prefix}")
    print(f"Tracking server ARN: {tracking_server_arn}")
    
    pipeline_name = f"{pipeline_name_prefix}-{sagemaker_project_id}"
    experiment_name = pipeline_name

    output_s3_prefix = f"s3://{bucket_name}/{bucket_prefix}"
    # Set the output S3 url for model artifact
    output_s3_url = f"{output_s3_prefix}/output"
    # Set the output S3 url for feature store query results
    output_query_location = f'{output_s3_prefix}/offline-store/query_results'
    
    # Set the output S3 urls for processed data
    train_s3_url = f"{output_s3_prefix}/train"
    validation_s3_url = f"{output_s3_prefix}/validation"
    test_s3_url = f"{output_s3_prefix}/test"
    evaluation_s3_url = f"{output_s3_prefix}/evaluation"
    
    baseline_s3_url = f"{output_s3_prefix}/baseline"
    prediction_baseline_s3_url = f"{output_s3_prefix}/prediction_baseline"
    
    xgboost_image_uri = sagemaker.image_uris.retrieve(
            "xgboost", 
            region=region,
            version="1.5-1"
    )

    # If no tracking server ARN, try to find an active MLflow server
    if tracking_server_arn is None:
        r = sm.list_mlflow_tracking_servers(
            TrackingServerStatus='Created',
        )['TrackingServerSummaries']
    
        if len(r) < 1:
            print("You don't have any running MLflow servers. Exiting...")
            return None
        else:
            tracking_server_arn = r[0]['TrackingServerArn']
            print(f"Use the tracking server ARN:{tracking_server_arn}")
        
    # Parameters for pipeline execution
    
    # Set processing instance type
    process_instance_type_param = ParameterString(
        name="ProcessingInstanceType",
        default_value=process_instance_type,
    )

    # Set training instance type
    train_instance_type_param = ParameterString(
        name="TrainingInstanceType",
        default_value=train_instance_type,
    )

    # Set model approval param
    model_approval_status_param = ParameterString(
        name="ModelApprovalStatus",
        default_value="PendingManualApproval"
    )

    # Minimal threshold for model performance on the test dataset
    test_score_threshold_param = ParameterFloat(
        name="TestScoreThreshold", 
        default_value=test_score_threshold
    )

    # S3 url for the input dataset
    input_s3_url_param = ParameterString(
        name="InputDataUrl",
        default_value=input_s3_url if input_s3_url else "None",
    )

    # Feature group name for the input featureset
    feature_group_name_param = ParameterString(
        name="FeatureGroupName",
        default_value=feature_group_name if feature_group_name else "None",
    )
    
    # Model package group name
    model_package_group_name_param = ParameterString(
        name="ModelPackageGroupName",
        default_value=model_package_group_name,
    )

    # MLflow tracking server ARN
    tracking_server_arn_param = ParameterString(
        name="TrackingServerARN",
        default_value=tracking_server_arn,
    )
    
    # Define step cache config
    cache_config = CacheConfig(
        enable_caching=True,
        expire_after="P30d" # 30-day
    )

    # Construct the pipeline
    
    # Get datasets
    step_get_datasets = step(
            preprocess, 
            role=role,
            instance_type=process_instance_type_param,
            name=f"preprocess",
            keep_alive_period_in_seconds=3600,
    )(
        input_data_s3_path=input_s3_url_param,
        output_s3_prefix=output_s3_prefix,
        tracking_server_arn=tracking_server_arn_param,
        experiment_name=experiment_name,
        pipeline_run_name=ExecutionVariables.PIPELINE_EXECUTION_ID,
    ) if input_s3_url else step(
        prepare_datasets, 
        role=role,
        instance_type=process_instance_type_param,
        name=f"extract-featureset",
        keep_alive_period_in_seconds=3600,
    )(
        feature_group_name=feature_group_name_param,
        output_s3_prefix=output_s3_prefix,
        query_output_s3_path=output_query_location,
        tracking_server_arn=tracking_server_arn_param,
        experiment_name=experiment_name,
        pipeline_run_name=ExecutionVariables.PIPELINE_EXECUTION_ID,
    )
    
    # Instantiate an XGBoost estimator object
    estimator = sagemaker.estimator.Estimator(
        image_uri=xgboost_image_uri,
        role=role, 
        instance_type=train_instance_type_param,
        instance_count=1,
        output_path=output_s3_url,
        sagemaker_session=session,
        base_job_name=f"{pipeline_name}-train"
    )
    
    # Define algorithm hyperparameters
    estimator.set_hyperparameters(
        num_round=100, # the number of rounds to run the training
        max_depth=3, # maximum depth of a tree
        eta=0.5, # step size shrinkage used in updates to prevent overfitting
        alpha=2.5, # L1 regularization term on weights
        objective="binary:logistic",
        eval_metric="auc", # evaluation metrics for validation data
        subsample=0.8, # subsample ratio of the training instance
        colsample_bytree=0.8, # subsample ratio of columns when constructing each tree
        min_child_weight=3, # minimum sum of instance weight (hessian) needed in a child
        early_stopping_rounds=10, # the model trains until the validation score stops improving
        verbosity=1, # verbosity of printing messages
    )
    
    # train step
    step_train = TrainingStep(
        name=f"train",
        step_args=estimator.fit(
            {
                "train": TrainingInput(
                    step_get_datasets['train_data'],
                    content_type="text/csv",
                ),
                "validation": TrainingInput(
                    step_get_datasets['validation_data'],
                    content_type="text/csv",
                ),
            }
        ),
        cache_config=cache_config,
    )   
    
    # Evaluation step
    step_evaluate = step(
        evaluate,
        role=role,
        instance_type=process_instance_type_param,
        name=f"evaluate",
        keep_alive_period_in_seconds=3600,
    )(
        test_x_data_s3_path=step_get_datasets['test_x_data'],
        test_y_data_s3_path=step_get_datasets['test_y_data'],
        model_s3_path=step_train.properties.ModelArtifacts.S3ModelArtifacts,
        output_s3_prefix=output_s3_prefix,
        tracking_server_arn=tracking_server_arn_param,
        experiment_name=step_get_datasets['experiment_name'],
        pipeline_run_id=step_get_datasets['pipeline_run_id'],
    )

    # register model step
    step_register = step(
        register,
        role=role,
        instance_type=process_instance_type_param,
        name=f"register",
        keep_alive_period_in_seconds=3600,
    )(
        training_job_name=step_train.properties.TrainingJobName,
        model_package_group_name=model_package_group_name_param,
        model_approval_status=model_approval_status_param,
        evaluation_result=step_evaluate['evaluation_result'],
        output_s3_prefix=output_s3_url,
        tracking_server_arn=tracking_server_arn_param,
        experiment_name=step_get_datasets['experiment_name'],
        pipeline_run_id=step_get_datasets['pipeline_run_id'],
    )

    # fail the pipeline execution step
    step_fail = FailStep(
        name=f"fail",
        error_message=Join(on=" ", values=["Execution failed due to AUC Score < ", test_score_threshold_param]),
    )
    
    # condition to check in the condition step
    condition_gte = ConditionGreaterThanOrEqualTo(
            left=step_evaluate['evaluation_result']['classification_metrics']['auc_score']['value'],  
            right=test_score_threshold_param,
    )
    
    # conditional register step
    step_conditional_register = ConditionStep(
        name=f"check-metrics",
        conditions=[condition_gte],
        if_steps=[step_register],
        else_steps=[step_fail],
    )   

    # Create a pipeline object
    pipeline = Pipeline(
        name=f"{pipeline_name}",
        parameters=[
            input_s3_url_param,
            feature_group_name_param,
            process_instance_type_param,
            train_instance_type_param,
            model_approval_status_param,
            test_score_threshold_param,
            model_package_group_name_param,
            tracking_server_arn_param,
        ],
        steps=[step_conditional_register],
        pipeline_definition_config=PipelineDefinitionConfig(use_custom_job_prefix=True)
    )
    
    return pipeline

Overwriting pipeline.py


Copy this `pipeline.py` file from the workshop folder to the `pipelines/fromideatoprod` folder in the project's code repository folder:

In [196]:
!cp ~/{workshop_folder}/pipeline.py ~/{project_path}/pipelines/fromideatoprod/

Test the function `get_pipeline` locally to see if everything works before running as remotely.

In [197]:
from pipeline import get_pipeline

In [ ]:
# If you created a feature store in the notebook 3, you can set the feature_group_name parameter instead of input_s3_url to take the data from the feature store
p = get_pipeline(
    region=region,
    sagemaker_project_id=project_id,
    sagemaker_project_name=project_name,
    role=sm_role,
    bucket_name=bucket_name,
    bucket_prefix=bucket_prefix,
    input_s3_url=input_s3_url,
    # feature_group_name=dataset_feature_group_name,
    model_package_group_name=model_package_group_name,
    pipeline_name_prefix=pipeline_name,
    process_instance_type="ml.m5.large",
    train_instance_type="ml.m5.xlarge",
    test_score_threshold=0.70,
    tracking_server_arn=mlflow_arn,
)

In [ ]:
p.definition()

In [ ]:
p.upsert(role_arn=sm_role)

To see the created pipeline in the Studio UI, click on the link constructed by the code cell below:

In [201]:
from IPython.display import HTML

# Show the pipeline link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/pipelines/{}/graph">the pipeline</a> in the Studio UI</b>'.format(
            domain_id, region, p.describe()['PipelineName']))
)

At this point you have tested locally that the pipeline construction code works and it creates a pipeline. You can see this pipeline in Studio **Pipelines** widget. Now you ready to create a CI/CD pipeline.

이 시점에서 파이프라인 구성 코드가 작동하고 파이프라인을 생성한다는 것을 로컬에서 테스트했습니다. 이 파이프라인은 Studio의 **Pipelines** 위젯에서 확인할 수 있습니다. 이제 CI/CD 파이프라인을 생성할 준비가 되었습니다.

주요 확인 사항:
1. 로컬 테스트 완료
   - 파이프라인 구성 코드 검증
   - 파이프라인 생성 확인

2. Studio UI 확인
   - Pipelines 위젯에서 파이프라인 가시화
   - 구성 상태 확인 가능

3. 다음 단계 준비
   - CI/CD 파이프라인 구축 준비 완료
   - 자동화된 배포 프로세스 구현 가능

이제 검증된 코드를 기반으로 자동화된 CI/CD 파이프라인을 구축할 수 있습니다.

#### Control project ownership with resource tags
Project-owned resources are automatically tagged with `sagemaker:project-name` and `sagemaker:project-id` tags for cost control, attribute-based security control, and governance. 

If you need to attach an existing repository, pipeline, model registry group, or endpoint to the project, you can add these two tags with the project name and id to the resource.

For example, the next code cell adds `sagemaker:*` tags to the model package group you created in the previous notebook. As the result, the model package group is now visible as a project resource.

#### 리소스 태그로 프로젝트 소유권 제어
프로젝트 소유 리소스에는 비용 제어, 속성 기반 보안 제어 및 거버넌스를 위해 `sagemaker:project-name` 및 `sagemaker:project-id` 태그가 자동으로 지정됩니다.

기존 저장소, 파이프라인, 모델 레지스트리 그룹 또는 엔드포인트를 프로젝트에 연결해야 하는 경우, 이 두 태그를 프로젝트 이름과 ID와 함께 리소스에 추가할 수 있습니다.

예를 들어, 다음 코드 셀은 이전 노트북에서 생성한 모델 패키지 그룹에 `sagemaker:*` 태그를 추가합니다. 결과적으로 모델 패키지 그룹이 이제 프로젝트 리소스로 표시됩니다.

주요 내용:
1. 자동 태그 지정
   - sagemaker:project-name
   - sagemaker:project-id

2. 태그 활용 목적
   - 비용 제어
   - 보안 제어
   - 거버넌스 관리

3. 기존 리소스 연결
   - 수동 태그 추가 가능
   - 프로젝트와 리소스 연동

4. 리소스 가시성
   - 태그 추가로 프로젝트 리소스로 표시
   - 통합된 관리 가능

In [202]:
model_package_group_arn = sm.describe_model_package_group(ModelPackageGroupName=model_package_group_name).get("ModelPackageGroupArn")

if model_package_group_arn:
    print(f"Adding tags {project_arn.split('/')[-1]} and {project_id} for model package group {model_package_group_arn}")
    r = sm.add_tags(
        ResourceArn=model_package_group_arn,
        Tags=[
            {
                'Key': 'sagemaker:project-name',
                'Value': project_arn.split("/")[-1]
            },
            {
                'Key': 'sagemaker:project-id',
                'Value': project_id
            },
        ]
    )
    print(r)
else:
    print(f"The model package group {model_package_group_name} doesn't exist")
    
sm.list_tags(ResourceArn=model_package_group_arn)["Tags"]

Adding tags mlops-02-14-11-03-33 and p-ca7phmcraepa for model package group arn:aws:sagemaker:us-east-1:906545278380:model-package-group/from-idea-to-prod-pipeline-model-12-06-30-22
{'Tags': [{'Key': 'sagemaker:project-name', 'Value': 'mlops-02-14-11-03-33'}, {'Key': 'sagemaker:project-id', 'Value': 'p-ca7phmcraepa'}], 'ResponseMetadata': {'RequestId': 'be9be1bf-da7a-4cdf-944f-0ae71254b32a', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'be9be1bf-da7a-4cdf-944f-0ae71254b32a', 'content-type': 'application/x-amz-json-1.1', 'content-length': '130', 'date': 'Sun, 16 Feb 2025 21:18:54 GMT'}, 'RetryAttempts': 0}}


[{'Key': 'sagemaker:project-name', 'Value': 'mlops-02-14-11-03-33'},
 {'Key': 'sagemaker:project-id', 'Value': 'p-ca7phmcraepa'}]

### 3. Modify the build specification file
Now modify the `codebuild-buildspec.yml` file in the project folder to reflect the new name of the Python module with your pipeline and set other project-specific parameters.

### 3. 빌드 명세 파일 수정
이제 프로젝트 폴더의 `codebuild-buildspec.yml` 파일을 수정하여 파이프라인이 포함된 Python 모듈의 새 이름을 반영하고 다른 프로젝트별 매개변수를 설정합니다.

주요 수정 사항:
1. Python 모듈 이름 업데이트
   - 새로운 파이프라인 모듈 참조
   - 올바른 경로 설정

2. 프로젝트 매개변수 설정
   - 프로젝트 특정 설정 반영
   - 환경 변수 구성

3. 빌드 프로세스 설정
   - 빌드 단계 정의
   - 의존성 관리
   - 실행 환경 구성

이러한 수정을 통해 CI/CD 파이프라인이 올바르게 코드를 빌드하고 실행할 수 있게 됩니다.

You need to pass the following parameters to a pipeline creation script - corresponding to the parameters of the `get_pipeline` function you've just created:
- `input_s3_url` - an S3 URL for the input raw dataset. If you created a feature group in the notebook 3, you can use the parameter `feature_group_name` instead
- `feature_group_name` – you can use this parameter if you created this feature group in the notebook 3, in this case you don't need to provide `input_s3_url`
- `model_package_group_name` – the model registry package to register a model after training
- `pipeline_name_prefix` – a name prefix for the pipeline. The pipeline name is constructed as `<pipeline_name_prefix>-<project-id>`
- `role` – the pipeline execution role
- `tracking_server_arn`- the MLflow server ARN for pipeline execution tracking

The following cells print the values of these parameters:

방금 생성한 `get_pipeline` 함수의 매개변수에 해당하는 다음 매개변수들을 파이프라인 생성 스크립트에 전달해야 합니다:

- `input_s3_url` - 입력 원시 데이터셋의 S3 URL. 노트북 3에서 피처 그룹을 생성한 경우 대신 `feature_group_name` 매개변수를 사용할 수 있습니다
- `feature_group_name` - 노트북 3에서 이 피처 그룹을 생성한 경우 사용할 수 있으며, 이 경우 `input_s3_url`을 제공할 필요가 없습니다
- `model_package_group_name` - 학습 후 모델을 등록할 모델 레지스트리 패키지
- `pipeline_name_prefix` - 파이프라인의 이름 접두사. 파이프라인 이름은 `<pipeline_name_prefix>-<project-id>`로 구성됩니다
- `role` - 파이프라인 실행 역할
- `tracking_server_arn` - 파이프라인 실행 추적을 위한 MLflow 서버 ARN

다음 셀들은 이러한 매개변수들의 값을 출력합니다:

주요 포인트:
1. 데이터 소스 선택
   - S3 URL 또는 피처 그룹 사용
   - 상호 배타적 선택

2. 파이프라인 식별자
   - 고유한 이름 생성
   - 프로젝트 ID 포함

3. 실행 환경 설정
   - 실행 역할 지정
   - MLflow 추적 설정

4. 모델 관리
   - 모델 패키지 그룹 지정
   - 버전 관리 지원

In [203]:
try:
    print(f"""
        INPUT-S3-URL: {input_s3_url}
        FEATURE-GROUP-NAME: {dataset_feature_group_name}
        MODEL-PACKAGE-GROUP-NAME: {project_name}-{project_id}
        PIPELINE-NAME-PREFIX: {pipeline_name}
        ROLE: {sm_role}
        TRACKING-SERVER-ARN: {mlflow_arn}
        """)
except NameError:
    print(f"""
        Dataset feature group name is not defined, use input_s3_url instead:
        ********************************************************************
        
        INPUT-S3-URL: {input_s3_url}
        MODEL-PACKAGE-GROUP-NAME: {project_name}-{project_id}
        PIPELINE-NAME-PREFIX: {pipeline_name}
        ROLE: {sm_role}
        TRACKING-SERVER-ARN: {mlflow_arn}
        """)


        INPUT-S3-URL: s3://sagemaker-us-east-1-906545278380/from-idea-to-prod/xgboost/input/bank-additional-full.csv
        FEATURE-GROUP-NAME: from-idea-to-prod-12-06-30-53
        MODEL-PACKAGE-GROUP-NAME: mlops-02-14-11-03-33-p-ca7phmcraepa
        PIPELINE-NAME-PREFIX: from-idea-to-prod-pipeline-12-06-30-22
        ROLE: arn:aws:iam::906545278380:role/mlops-workshop-domain-SageMakerExecutionRole-rIgas55nwmQD
        TRACKING-SERVER-ARN: arn:aws:sagemaker:us-east-1:906545278380:mlflow-tracking-server/test-mlflow
        


Now replace the value of these parameters in the following code cell with the printed values from the cell above. 

To do it, locate the parameter `kwargs` in the following code cell starting with `%%writefile codebuild-buildspec.yml`:

```
--kwargs "{ \
    \"input_s3_url\":\"<INPUT-S3-URL>\", \
    \"feature_group_name\":\"<FEATURE-GROUP-NAME>\", \
    \"model_package_group_name\":\"<MODEL-PACKAGE-GROUP-NAME>\", \
    \"pipeline_name_prefix\":\"<PIPELINE-BASE-NAME>\", \
    \"role\":\"<SAGEMAKER-EXECUTION-ROLE-ARN>\", \
    \"tracking_server_arn\":\"<TRACKING-SERVER-ARN>\", \
    \"region\":\"${AWS_REGION}\", \
    \"sagemaker_project_name\":\"${SAGEMAKER_PROJECT_NAME}\", \
    \"sagemaker_project_id\":\"${SAGEMAKER_PROJECT_ID}\", \
    \"bucket_name\":\"${ARTIFACT_BUCKET}\" \
        }"
```

and replace the values of `input_s3_url` OR `feature_group_name`, `model_package_group_name`, `pipeline_name_prefix`, `role`, and `tracking_server_arn` parameters with the values printed by the previous cells.

You need to replace only one of `input_s3_url` or `feature_group_name` depending on what dataset input method you'd like to use - a raw input dataset from S3 or the processed featureset from the feature store. You can use the feature store only if you created it in the previous notebook. 

<div class="alert alert-info">Delete the line with parameter that you don't use: <code>input_s3_url</code> or <code>feature_group_name</code> from the cell code.</div>

![](img/codebuild-buildspec-edit.png)

After you replace the values of the parameter, execute the cell to write a build spec file.

이제 위 셀에서 출력된 값으로 다음 코드 셀의 이러한 매개변수 값을 교체하세요.

`%%writefile codebuild-buildspec.yml`로 시작하는 다음 코드 셀에서 `kwargs` 매개변수를 찾으세요:

```
--kwargs "{ \
    \"input_s3_url\":\"<INPUT-S3-URL>\", \
    \"feature_group_name\":\"<FEATURE-GROUP-NAME>\", \
    \"model_package_group_name\":\"<MODEL-PACKAGE-GROUP-NAME>\", \
    \"pipeline_name_prefix\":\"<PIPELINE-BASE-NAME>\", \
    \"role\":\"<SAGEMAKER-EXECUTION-ROLE-ARN>\", \
    \"tracking_server_arn\":\"<TRACKING-SERVER-ARN>\", \
    \"region\":\"${AWS_REGION}\", \
    \"sagemaker_project_name\":\"${SAGEMAKER_PROJECT_NAME}\", \
    \"sagemaker_project_id\":\"${SAGEMAKER_PROJECT_ID}\", \
    \"bucket_name\":\"${ARTIFACT_BUCKET}\" \
        }"
```

이전 셀에서 출력된 값으로 `input_s3_url` 또는 `feature_group_name`, `model_package_group_name`, `pipeline_name_prefix`, `role`, `tracking_server_arn` 매개변수의 값을 교체하세요.

사용하고자 하는 데이터셋 입력 방식에 따라 `input_s3_url` 또는 `feature_group_name` 중 하나만 교체하면 됩니다 - S3의 원시 입력 데이터셋 또는 피처 스토어의 처리된 피처셋. 이전 노트북에서 생성한 경우에만 피처 스토어를 사용할 수 있습니다.

<div class="alert alert-info">사용하지 않는 매개변수의 줄을 셀 코드에서 삭제하세요: <code>input_s3_url</code> 또는 <code>feature_group_name</code></div>

![](img/codebuild-buildspec-edit.png)

매개변수 값을 교체한 후 셀을 실행하여 빌드 명세 파일을 작성하세요.

주요 작업:
1. 매개변수 값 교체
2. 불필요한 매개변수 라인 삭제
3. 빌드 명세 파일 생성
4. 구성 확인

In [204]:
%%writefile codebuild-buildspec.yml

version: 0.2

phases:
  install:
    runtime-versions:
      python: 3.10
    commands:
      - pip install --upgrade --force-reinstall . "awscli>1.20.30"
      - pip install --upgrade mlflow sagemaker-mlflow s3fs xgboost
    
  build:
    commands:
      - export SAGEMAKER_USER_CONFIG_OVERRIDE="./config.yaml"
      - export PYTHONUNBUFFERED=TRUE
      - export SAGEMAKER_PROJECT_NAME_ID="${SAGEMAKER_PROJECT_NAME}-${SAGEMAKER_PROJECT_ID}"
      - |
        run-pipeline --module-name pipelines.fromideatoprod.pipeline \
          --role-arn $SAGEMAKER_PIPELINE_ROLE_ARN \
          --tags "[{\"Key\":\"sagemaker:project-name\",\"Value\":\"${SAGEMAKER_PROJECT_NAME}\"}, {\"Key\":\"sagemaker:project-id\", \"Value\":\"${SAGEMAKER_PROJECT_ID}\"}]" \
          --kwargs "{ \
                \"input_s3_url\":\"s3://sagemaker-us-east-1-906545278380/from-idea-to-prod/xgboost/input/bank-additional-full.csv\", \
                \"model_package_group_name\":\"mlops-02-14-11-03-33-p-ca7phmcraepa\",\
                \"pipeline_name_prefix\":\"from-idea-to-prod-pipeline-12-06-30-22\",\
                \"role\":\"arn:aws:iam::906545278380:role/mlops-workshop-domain-SageMakerExecutionRole-rIgas55nwmQD\",\
                \"tracking_server_arn\":\"arn:aws:sagemaker:us-east-1:906545278380:mlflow-tracking-server/test-mlflow\", \
                \"region\":\"${AWS_REGION}\", \
                \"sagemaker_project_name\":\"${SAGEMAKER_PROJECT_NAME}\",\
                \"sagemaker_project_id\":\"${SAGEMAKER_PROJECT_ID}\",\
                \"bucket_name\":\"${ARTIFACT_BUCKET}\"\
                    }"
      - echo "Create/update of the SageMaker Pipeline and a pipeline execution completed."

Overwriting codebuild-buildspec.yml


Copy the `codebuild-buildspec.yml` file from the workshop folder to the project's code repository folder:

In [205]:
!cp ~/{workshop_folder}/codebuild-buildspec.yml ~/{project_path}/codebuild-buildspec.yml

To summarize, you have just done three changes in the build spec file:
1. Modified the `run-pipeline` `--module-name` parameter value from `pipelines.abalone.pipeline` to the new path `pipelines.fromideatoprod.pipeline`
2. Removed some parameters from the `kwargs` list to make use of `get_pipeline()` function default parameter values
3. Added additional parameters for the pipeline to the `kwargs` parameter list

요약하면, 빌드 명세 파일에서 다음 세 가지 변경을 수행했습니다:
1. `run-pipeline` `--module-name` 매개변수 값을 `pipelines.abalone.pipeline`에서 새로운 경로 `pipelines.fromideatoprod.pipeline`로 수정
2. `get_pipeline()` 함수의 기본 매개변수 값을 사용하기 위해 `kwargs` 목록에서 일부 매개변수 제거
3. 파이프라인을 위한 추가 매개변수를 `kwargs` 매개변수 목록에 추가

주요 변경사항:
1. 모듈 경로 업데이트
   - 기존 경로 변경
   - 새로운 파이프라인 모듈 참조

2. 매개변수 최적화
   - 불필요한 매개변수 제거
   - 기본값 활용

3. 파이프라인 구성 확장
   - 새로운 매개변수 추가
   - 기능 확장 지원

이러한 변경을 통해 파이프라인이 더 효율적이고 유연하게 구성되었습니다.

### 4. Create the `setup.py` file
Finally, you need to provide the `setup.py` file in the project's code repository folder.

### 4. `setup.py` 파일 생성
마지막으로, 프로젝트의 코드 저장소 폴더에 `setup.py` 파일을 제공해야 합니다.

주요 내용:
1. Python 패키지 설정
   - 프로젝트 메타데이터 정의
   - 의존성 관리
   - 설치 구성

2. 파일의 역할
   - 파이프라인 코드 패키징
   - 필요한 종속성 정의
   - 배포 설정 관리

3. 설치 구성
   - 필요한 라이브러리 지정
   - 버전 관리
   - 패키지 구조 정의

이 파일은 프로젝트의 Python 패키지 구성과 배포를 관리하는 중요한 역할을 합니다.

In [206]:
%%writefile setup.py
import os
import setuptools


about = {}
here = os.path.abspath(os.path.dirname(__file__))
with open(os.path.join(here, "pipelines", "__version__.py")) as f:
    exec(f.read(), about)


with open("README.md", "r") as f:
    readme = f.read()


required_packages = ["sagemaker"]
extras = {
    "test": [
        "black",
        "coverage",
        "flake8",
        "mock",
        "pydocstyle",
        "pytest",
        "pytest-cov",
        "sagemaker",
        "tox",
    ]
}
setuptools.setup(
    name=about["__title__"],
    description=about["__description__"],
    version=about["__version__"],
    author=about["__author__"],
    author_email=["__author_email__"],
    long_description=readme,
    long_description_content_type="text/markdown",
    url=about["__url__"],
    license=about["__license__"],
    packages=setuptools.find_packages(),
    include_package_data=True,
    python_requires=">=3.6",
    install_requires=required_packages,
    extras_require=extras,
    entry_points={
        "console_scripts": [
            "get-pipeline-definition=pipelines.get_pipeline_definition:main",
            "run-pipeline=pipelines.run_pipeline:main",
        ]
    },
    classifiers=[
        "Development Status :: 3 - Alpha",
        "Intended Audience :: Developers",
        "Natural Language :: English",
        "Programming Language :: Python",
        "Programming Language :: Python :: 3",
        "Programming Language :: Python :: 3.6",
        "Programming Language :: Python :: 3.7",
        "Programming Language :: Python :: 3.8",
    ],
)

Overwriting setup.py


In [207]:
!cp ~/{workshop_folder}/setup.py ~/{project_path}/setup.py

---

## Run the CI/CD for the model building pipeline
To launch the CI/CD for the model building pipeline you need to push the changed code into the project GitHub repository.

<div class="alert alert-info">Make sure you are in the folder that contains the repository code in JupyterLab terminal when running git commands. The folder name looks like <code>[project-name]/sagemaker-[project-id]-modelbuild</code>.</div>

Open a system terminal window via the JupyterLab menu **File** > **New** > **Terminal** and enter the commands generated by the following code cell.
Keep `user.email` and `user.name` or replace with your data.

## 모델 구축 파이프라인의 CI/CD 실행
모델 구축 파이프라인의 CI/CD를 시작하려면 변경된 코드를 프로젝트 GitHub 저장소에 푸시해야 합니다.

<div class="alert alert-info">git 명령을 실행할 때 JupyterLab 터미널에서 저장소 코드가 있는 폴더에 있는지 확인하세요. 폴더 이름은 <code>[project-name]/sagemaker-[project-id]-modelbuild</code>와 같은 형식입니다.</div>

JupyterLab 메뉴 **File** > **New** > **Terminal**을 통해 시스템 터미널 창을 열고 다음 코드 셀에서 생성된 명령을 입력하세요.
`user.email`과 `user.name`을 유지하거나 사용자의 데이터로 교체하세요.

주요 단계:
1. 터미널 접근
   - JupyterLab 메뉴 사용
   - 올바른 폴더 확인

2. Git 설정
   - 사용자 이메일 설정
   - 사용자 이름 설정

3. 코드 푸시
   - 변경사항 커밋
   - GitHub 저장소에 푸시

4. CI/CD 시작
   - 자동화된 파이프라인 트리거
   - 빌드 프로세스 시작

In [188]:
cmd = f'''
cd ~/{project_path}

git config --global user.email "you@example.com"
git config --global user.name "Your Name"
  
git add -A
git commit -am "customize project"
git push
'''

copy_output(cmd)

After pushing your code changes, the project initiates a run of the CodePipeline pipeline that constructs, upcerts, and executes the SageMaker model building pipeline. This new pipeline execution creates a new model version in the model package group in the SageMaker model registry.

You can follow up the execution of the pipeline in the Studio **Pipelines** widget.

Wait until the pipeline execution finishes. The execution takes about 10 minutes to complete.

코드 변경사항을 푸시한 후, 프로젝트는 SageMaker 모델 구축 파이프라인을 구성, 업서트(upsert)하고 실행하는 CodePipeline 파이프라인 실행을 시작합니다. 이 새로운 파이프라인 실행은 SageMaker 모델 레지스트리의 모델 패키지 그룹에 새로운 모델 버전을 생성합니다.

Studio의 **Pipelines** 위젯에서 파이프라인 실행 상황을 추적할 수 있습니다.

파이프라인 실행이 완료될 때까지 기다리세요. 실행은 약 10분이 소요됩니다.

주요 단계:
1. 자동화된 프로세스 시작
   - 코드 푸시로 트리거
   - CodePipeline 실행 시작

2. 파이프라인 작업
   - 모델 구축 파이프라인 구성
   - 파이프라인 업서트
   - 자동 실행

3. 모델 버전 관리
   - 새 모델 버전 생성
   - 모델 레지스트리 업데이트

4. 모니터링
   - Pipelines 위젯으로 진행상황 추적
   - 약 10분 소요 예상

To see the execution of the pipeline click on the link constructed by the code cell below. Note, that CodeBuild takes about 1 minute to build upsert the pipeline and to start the execution. Refresh the Studio UI page to see the started execution.

아래 코드 셀에서 생성된 링크를 클릭하여 파이프라인 실행을 확인하세요. CodeBuild가 파이프라인을 업서트하고 실행을 시작하는 데 약 1분이 소요됩니다. 시작된 실행을 보려면 Studio UI 페이지를 새로고침하세요.

주요 안내:
1. 실행 확인 방법
   - 제공된 링크 클릭
   - Studio UI에서 확인

2. 소요 시간
   - CodeBuild 초기 설정: 약 1분
   - 파이프라인 업서트
   - 실행 시작

3. 화면 갱신
   - Studio UI 새로고침 필요
   - 실행 상태 업데이트 확인

4. 모니터링 포인트
   - 파이프라인 구성 상태
   - 실행 진행 상황
   - 각 단계별 상태

In [190]:
# Show the pipeline execution link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/pipelines/{}/executions/">the pipeline executions</a> in the Studio UI</b>'.format(
            domain_id, region, p.describe()['PipelineName']))
)

## View the model package in the model registry

This new pipeline creates a new model package group named `<project_name>-<project-id` in the SageMaker model registry.

To see the model package version in the Studio UI click on the link constructed by the code cell below. 
<div class="alert alert-info">You need to wait until the pipeline execution finishes to see a registered version of the model in the model package.</div>

On the model version tab that opens, you can browse activity, [model version details](https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry-details.html), and [data lineage](https://docs.aws.amazon.com/sagemaker/latest/dg/lineage-tracking.html). 

In a real-world project you add various model attributes and additional model version metadata such as [model quality metrics](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-model-quality-metrics.html), [explainability](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-model-explainability.html) and [bias](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-measure-post-training-bias.html) reports, load test data, and [inference recommender](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-recommender.html).

## 모델 레지스트리에서 모델 패키지 보기

이 새로운 파이프라인은 SageMaker 모델 레지스트리에 `<project_name>-<project-id>` 이름의 새 모델 패키지 그룹을 생성합니다.

Studio UI에서 모델 패키지 버전을 보려면 아래 코드 셀에서 생성된 링크를 클릭하세요.
<div class="alert alert-info">모델 패키지에서 등록된 모델 버전을 보려면 파이프라인 실행이 완료될 때까지 기다려야 합니다.</div>

열리는 모델 버전 탭에서 활동, [모델 버전 세부 정보](https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry-details.html), [데이터 계보](https://docs.aws.amazon.com/sagemaker/latest/dg/lineage-tracking.html)를 탐색할 수 있습니다.

실제 프로젝트에서는 다음과 같은 다양한 모델 속성과 추가 모델 버전 메타데이터를 추가합니다:
- [모델 품질 메트릭](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-model-quality-metrics.html)
- [설명 가능성](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-model-explainability.html)
- [편향성](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-measure-post-training-bias.html) 보고서
- 부하 테스트 데이터
- [추론 추천기](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-recommender.html)

주요 내용:
1. 모델 패키지 확인
   - 새로 생성된 패키지 그룹 확인
   - 버전 정보 확인

2. 모니터링 요소
   - 활동 로그
   - 버전 세부 정보
   - 데이터 계보

3. 추가 가능한 메타데이터
   - 품질 메트릭
   - 설명 가능성
   - 편향성 분석
   - 성능 테스트 결과

4. 실제 운영 고려사항
   - 종합적인 모델 관리
   - 다양한 메트릭 추적
   - 품질 보증 요소

In [191]:
# Show the model package link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/models/registered-models/{}-{}/versions">the model package versions</a> in the Studio UI</b>'.format(
            domain_id, region, project_name, project_id))
)

## Summary
In this notebook you implement a CI/CD pipeline with the following features:
- A model building ML pipeline is under the source control in a GitHub repository
- Every push into the code repository launches a new CodePipeline pipiline which constructs, upserts, and executes the ML pipeline
- The whole end to end model development process is automated now
- SageMaker project is a logical construct in Studio which has the metadata about related ML pipelines, repositories, models, experiments, and inference endpoints

---

## 요약
이 노트북에서는 다음과 같은 기능을 가진 CI/CD 파이프라인을 구현했습니다:
- GitHub 저장소에서 소스 제어 하에 있는 모델 구축 ML 파이프라인
- 코드 저장소에 대한 모든 푸시가 ML 파이프라인을 구성, 업서트, 실행하는 새로운 CodePipeline 파이프라인을 시작
- 전체 엔드 투 엔드 모델 개발 프로세스가 이제 자동화됨
- SageMaker 프로젝트는 관련 ML 파이프라인, 저장소, 모델, 실험, 추론 엔드포인트에 대한 메타데이터를 가진 Studio의 논리적 구성체

주요 성과:
1. 자동화된 개발 환경
   - 소스 코드 버전 관리
   - 자동화된 파이프라인 실행
   - 지속적 통합/배포

2. 통합된 ML 워크플로우
   - 코드 변경에 따른 자동 업데이트
   - 일관된 모델 개발 프로세스
   - 효율적인 리소스 관리

3. 중앙화된 프로젝트 관리
   - 메타데이터 통합 관리
   - 컴포넌트 간 연계성
   - 종합적인 모니터링

4. MLOps 베스트 프랙티스
   - 자동화된 품질 관리
   - 추적 가능한 개발 과정
   - 확장 가능한 인프라 구조

## Continue with the step 5
In the next notebook [05-deploy](05-deploy.ipynb) you test the second part of the MLOps project - the model deployment pipeline.

## Further development ideas for your real-world projects
- You can use a SageMaker-provided [MLOps template for model building, training, and deployment with third-party Git repositories using Jenkins](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-templates-sm.html#sagemaker-projects-templates-git-jenkins)
- Create a [custom SageMaker project template](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-templates-custom.html) to cover your specific project requirements

## Additional resources
- [Amazon SageMaker Pipelines lab in SageMaker Immersion Day](https://catalog.us-east-1.prod.workshops.aws/workshops/63069e26-921c-4ce1-9cc7-dd882ff62575/en-US/lab6)
- [Enhance your machine learning development by using a modular architecture with Amazon SageMaker projects](https://aws.amazon.com/blogs/machine-learning/enhance-your-machine-learning-development-by-using-a-modular-architecture-with-amazon-sagemaker-projects/)
- [Dive deep into automating MLOps](https://www.youtube.com/watch?v=3_cHnk9VSfQ)
- [SageMaker MLOps Project Walkthrough](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-walkthrough.html)
- [`aws-samples` GitHub repository with custom project templates examples](https://github.com/aws-samples/sagemaker-custom-project-templates)

# Shutdown kernel

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>